In [10]:
import pandas as pd
import numpy as np
import torch
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler  # Importer StandardScaler

# Chargement des données
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

X = data_train.drop('SalePrice', axis=1).values  # On dégage la colonne SalePrice
y = data_train['SalePrice'].values

# Extraire les noms de colonnes à partir de data_train (avant de retirer 'SalePrice')
column_names = data_train.drop('SalePrice', axis=1).columns

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convertir X_train et X_test en DataFrame pour utiliser pd.get_dummies
X_train_df = pd.DataFrame(X_train, columns=column_names)
X_test_df = pd.DataFrame(X_test, columns=column_names)

#METHODE A : ON GARDE TOUT CE QUI A UNE BONNE CORRELATION
CoefCorrelLimit = 0.7

# Liste des colonnes qu'on veut garder en valeur numérique : 
numeric_columns = ['OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','YrSold']

# On convertit les colonnes numériques interprétées comme 'object' en types numériques explicitement
X_train_df[numeric_columns] = X_train_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Appliquer one-hot encoding sur toutes les colonnes catégorielles de X_train_df
Xa_train_df_encoded = pd.get_dummies(X_train_df, drop_first=True)
Xa_test_df_encoded = pd.get_dummies(X_test_df, drop_first=True)

# Alignement des colonnes de Xa_test_df_encoded avec celles de Xa_train_df_encoded
Xa_test_df_encoded = Xa_test_df_encoded.reindex(columns=Xa_train_df_encoded.columns, fill_value=0)

# Calculer la corrélation entre chaque feature et la cible (prix)
correlation = Xa_train_df_encoded.corrwith(pd.Series(y_train))  # y_train est la cible (prix)

# Filtrer la corrélation pour ne garder que celles avec une valeur absolue > CoefCorrelLimit
filtered_correlation = correlation[correlation.abs() > CoefCorrelLimit]

# Extraire les noms des colonnes sélectionnées à partir de la corrélation filtrée
selected_columns = filtered_correlation.index

# Filtrer les données d'entraînement et de test pour ne garder que les colonnes sélectionnées
Xa_train_selected = Xa_train_df_encoded[selected_columns]
Xa_test_selected = Xa_test_df_encoded[selected_columns]

# STANDARDISATION DES DONNÉES
scaler = StandardScaler()

# Ajuster le scaler sur les données d'entraînement et les transformer
Xa_train_selected_scaled = scaler.fit_transform(Xa_train_selected)

# Transformer les données de test en utilisant le même scaler
Xa_test_selected_scaled = scaler.transform(Xa_test_selected)

# Création et entraînement du modèle
model = LinearRegression()

# Supprimer les lignes contenant des NaN dans Xa_train_selected_scaled et Xa_test_selected_scaled
Xa_train_selected_scaled = pd.DataFrame(Xa_train_selected_scaled, columns=selected_columns)
Xa_test_selected_scaled = pd.DataFrame(Xa_test_selected_scaled, columns=selected_columns)

Xa_train_selected_scaled = Xa_train_selected_scaled.dropna()
Xa_test_selected_scaled = Xa_test_selected_scaled.dropna()

# Assurez-vous que y_train et y_test soient également synchronisés avec les données filtrées
y_train = y_train[Xa_train_selected_scaled.index]
y_test = y_test[Xa_test_selected_scaled.index]

# Entraîner le modèle avec les données d'entraînement standardisées
model.fit(Xa_train_selected_scaled, y_train)

# Faire des prédictions sur les données de test
y_pred = model.predict(Xa_test_selected_scaled)

# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Afficher les résultats
print(f'Mean Squared Error: {mse}')
print(f'R²: {r2}')

comparison_df = pd.DataFrame({
    'Y Réel': y_test,    # Valeurs réelles (targets)
    'Y Prédit': y_pred   # Valeurs prédites
})

# Afficher les premières lignes du tableau
print(comparison_df.head())

# Réinitialiser l'option pour l'affichage des lignes
pd.reset_option('display.max_rows')


Mean Squared Error: 86114496684.58127
R²: -16.170024296145385
   Y Réel       Y Prédit
0  158000 -109082.227524
1  175900 -109082.227524
2   92000 -109082.227524
3  140000 -109082.227524
4  326000 -109082.227524


In [59]:
#METHODE B : SELECTION MANUELLE DES MEILLEURES FEATURES

import pandas as pd
import numpy as np
import torch
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler  # Importer StandardScaler

# Chargement des données
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Liste des colonnes/features à tester
features = ['OverallQual', 'TotRmsAbvGrd', 'LotArea', 'GrLivArea', 'YrSold']
target = 'SalePrice'
nbEssais = 30

# Liste des combinaisons de 2, 3, 4 et 5 features
combinations = []
for r in range(2, 6):  # De 2 à 5 features
    combinations.extend(itertools.combinations(features, r))

# Dictionnaire pour stocker les résultats
results = []

# Entraînement et évaluation pour chaque combinaison de features
for comb in combinations:
    print(f"Entraînement pour la combinaison : {comb}")
    
    errors = []  # Liste pour stocker l'erreur de chaque essai (30 essais)
    
    for _ in range(nbEssais):  # Faire nbEssais essais
        # Sélectionner les données correspondant à la combinaison de features
        X = data_train[list(comb)].values  # Sélectionner les features de cette combinaison
        y = data_train[target].values
        
        # Diviser les données en train et test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        
        # Standardisation des données
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Création et entraînement du modèle
        model = LinearRegression()
        model.fit(X_train_scaled, y_train)
        
        # Faire des prédictions sur les données de test
        y_pred = model.predict(X_test_scaled)
        
        # Calcul de l'erreur quadratique moyenne
        mse = mean_squared_error(y_test, y_pred)
        
        # Calcul de l'erreur en pourcentage
        mean_error = 100 * (mse ** 0.5) * len(y_test) / sum(y_test)
        errors.append(mean_error)
    
    # Calculer l'erreur moyenne sur les 30 essais
    mean_error = np.mean(errors)
    
    # Sauvegarder les résultats
    results.append((comb, mean_error))

# Trier les résultats par erreur moyenne croissante
results.sort(key=lambda x: x[1])

# Afficher les résultats
print("\nClassement des combinaisons par erreur moyenne :")
for comb, mean_error in results:
    print(f"Combinaison {comb} - Erreur moyenne : {mean_error:.2f}%")

#On utilise la meilleure combinaison, et on formatte les resultats après l'avoir appliquée sur les données de test : 

# Sélectionner la meilleure combinaison
best_combination = results[0][0]  # La première combinaison (la meilleure) après le tri

# Charger le fichier test.csv pour faire des prédictions
data_test = pd.read_csv('test.csv')

# Sélectionner les données correspondantes à la meilleure combinaison
X_test_best = data_test[list(best_combination)].values

# Standardisation des données de test avec le même scaler que pour l'entraînement
scaler_best = StandardScaler()
X_test_best_scaled = scaler_best.fit_transform(X_test_best)

# Créer un modèle et l'entraîner avec les données d'entraînement et la meilleure combinaison de features
X_train_best = data_train[list(best_combination)].values
y_train_best = data_train[target].values

# Diviser les données d'entraînement en train et test pour l'entraînement du modèle
X_train_best_scaled = scaler_best.fit_transform(X_train_best)

# Création et entraînement du modèle
best_model = LinearRegression()
best_model.fit(X_train_best_scaled, y_train_best)

# Faire des prédictions sur les données de test
y_test_pred = best_model.predict(X_test_best_scaled)

# Créer un DataFrame avec l'ID et les résultats prédits
predictions_df = pd.DataFrame({
    'Id': data_test['Id'],  # Garder la colonne 'Id' du fichier original
    'SalePrice': y_test_pred  # Ajouter la colonne 'SalePrice' avec les prédictions
})

# Sauvegarder le DataFrame dans un fichier CSV avec les deux colonnes
predictions_df.to_csv('test_predictions.csv', index=False)

print("Les prédictions ont été sauvegardées dans 'test_predictions.csv'.")

print(len(predictions_df))


Entraînement pour la combinaison : ('OverallQual', 'TotRmsAbvGrd')
Entraînement pour la combinaison : ('OverallQual', 'LotArea')
Entraînement pour la combinaison : ('OverallQual', 'GrLivArea')
Entraînement pour la combinaison : ('OverallQual', 'YrSold')
Entraînement pour la combinaison : ('TotRmsAbvGrd', 'LotArea')
Entraînement pour la combinaison : ('TotRmsAbvGrd', 'GrLivArea')
Entraînement pour la combinaison : ('TotRmsAbvGrd', 'YrSold')
Entraînement pour la combinaison : ('LotArea', 'GrLivArea')
Entraînement pour la combinaison : ('LotArea', 'YrSold')
Entraînement pour la combinaison : ('GrLivArea', 'YrSold')
Entraînement pour la combinaison : ('OverallQual', 'TotRmsAbvGrd', 'LotArea')
Entraînement pour la combinaison : ('OverallQual', 'TotRmsAbvGrd', 'GrLivArea')
Entraînement pour la combinaison : ('OverallQual', 'TotRmsAbvGrd', 'YrSold')
Entraînement pour la combinaison : ('OverallQual', 'LotArea', 'GrLivArea')
Entraînement pour la combinaison : ('OverallQual', 'LotArea', 'YrSold'